In [1]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
import chromedriver_autoinstaller
import requests
import time
import re

In [4]:
class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
        #options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(10)
        driver.set_window_size(1600, 1100)

        return driver

In [9]:
driver = Driver.driver_init()
article_list = []
driver.get('https://policy.trade.ec.europa.eu/news_en?f%5B0%5D=trade_topics_trade_topics%3A104')
while True:
    current_url = driver.current_url
    articles = driver.find_elements('xpath', '//div[@class="ecl-content-block ecl-content-item__content-block"]//a')
    article_urls = [i.get_attribute('href') for i in articles]
    article_list.extend(article_urls)
    try:
        next = driver.find_element('xpath', '//ul[@class="ecl-pagination__list"]/li[@class="ecl-pagination__item ecl-pagination__item--next"]/a')
        next_url = next.get_attribute('href')
        driver.get(next_url)
    except:
        break


In [10]:
len(article_list)

97

In [19]:
for idx, article in enumerate(article_list):
    driver.get(article)
    current_url = driver.current_url
    if 'ted' in current_url:
        continue
    date = driver.find_element('xpath', '//div[@class="ecl-page-header__meta"]//span[2]').get_attribute('innerText')
    title = driver.find_element('xpath', '//div[contains(@class, "ecl-page-header__title")]//h1').get_attribute('innerText')
    #header_desc = driver.find_element('xpath', '//div[@class="ecl-page-header__description-container"]/p').get_attribute('innerText')
    try:
        main = driver.find_element('xpath', '//div[@id="block-ewcms-theme-main-page-content"]/article/div').get_attribute('innerText')
    except:
        main = driver.find_element('xpath', '//div[@class="ecl-paragraph"]').get_attribute('innerText')
    file_name = f'EU_commission_{idx}.txt'

    with open(file_name, 'w', encoding = 'utf-8') as file:
        file.write('<Date>' + '\n')
        file.write(date + '\n')
        file.write('<Title>' + '\n')
        file.write(title + '\n')
        file.write('<Content>' + '\n')
        #file.write(header_desc + '\n')
        file.write(main)
